In [131]:
####Either do this or go down to the "Read preprocessed files"

# Installing Dependencies

In [1]:
!pip install pytesseract 
!pip install pdf2image
!apt-get install poppler-utils 
!sudo apt install tesseract-ocr

  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13945 sha256=47af67137aa4fb0026114036efb91ac1bbc346ec5dce27136d1e5c9fb36bf3f7
  Stored in directory: /root/.cache/pip/wheels/81/20/7e/1dd0daad1575d5260916bb1e9781246430647adaef4b3ca3b3
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 0s (310 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 160975 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-

In [2]:
!pip install tika
!pip install networkx

  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32885 sha256=3c9d18d03371a42887affb1918564a4ecb74291b0012f018876b4c77a0541cc6
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


# Importing Libraries

In [3]:
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 
from tika import parser
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
import spacy
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Reading File and Converting to Text

In [128]:
# Path of the pdf 
PDF_file = "/content/2021-02-08_104743.pdf"

pages = convert_from_path(PDF_file) 
rawText = ""

for page in pages:
  text = str(((pytesseract.image_to_string(page))))
  text = text.replace('-\n', '')
  rawText += text


 


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2766: DecompressionBombWarning: Image size (150876400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [129]:
print(rawText)
with open("output.txt", "w") as f:
  f.write(rawText)

For the use of a Registered Medical Practitioner or a Hospital or a Laboratory only.

Prescribing Information

Inandronic Acid Tablets
Idrofos 150

Dosage form:
Tablets

Composition:

Each Film Coated Tablet Contains:

lbandronic Acid Monosodium Monohydrate equivalentto Ibandronic Acid 150 mg
Excipients on

Clinical Pharmacology:
lbandronic acid is anitrogen containing bisphosphonate that inhibits osteoclast mediated bone resorption.

Mechanism of Action

The action of ibandronic acid on bone tissue Is based on its affinity for hydroxyapatite, which is part of the
mineral matrix of bone. Ibandronic acid inhibits osteoclast activity and reduces bone resorption and turnover. In
post menopausal women, it reduces the elevated rate of bone turnover, leading to, on average, a net gain in
bone mass.

Pharmacokinetics

The absorption of oral ibandronic acid occurs in the upper gastrointestinal tract. Plasma concentrations
increase ina dose linear manner up to 50 mg oral intake and increases no

# PreProcessing

In [6]:
rawList = rawText.splitlines()

In [7]:
rawList = [item for item in rawList if item.strip()]

In [8]:
len(rawList)

198

In [9]:
rawList_str = ' '.join(rawList)
print(rawList_str)

For the use of a Registered Medical Practitioner or a Hospital or a Laboratory only. Prescribing Information Inandronic Acid Tablets Idrofos 150 Dosage form: Tablets Composition: Each Film Coated Tablet Contains: lbandronic Acid Monosodium Monohydrate equivalentto Ibandronic Acid 150 mg Excipients on Clinical Pharmacology: lbandronic acid is anitrogen containing bisphosphonate that inhibits osteoclast mediated bone resorption. Mechanism of Action The action of ibandronic acid on bone tissue Is based on its affinity for hydroxyapatite, which is part of the mineral matrix of bone. Ibandronic acid inhibits osteoclast activity and reduces bone resorption and turnover. In post menopausal women, it reduces the elevated rate of bone turnover, leading to, on average, a net gain in bone mass. Pharmacokinetics The absorption of oral ibandronic acid occurs in the upper gastrointestinal tract. Plasma concentrations increase ina dose linear manner up to 50 mg oral intake and increases nonlinearly a

In [10]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in rawList_str.splitlines():
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

In [11]:
print(sentences[:5])

['For the use of a Registered Medical Practitioner or a Hospital or a Laboratory only.', 'Prescribing Information Inandronic Acid Tablets Idrofos 150 Dosage form: Tablets Composition: Each Film Coated Tablet Contains: lbandronic Acid Monosodium Monohydrate equivalentto Ibandronic Acid 150 mg Excipients on Clinical Pharmacology: lbandronic acid is anitrogen containing bisphosphonate that inhibits osteoclast mediated bone resorption.', 'Mechanism of Action The action of ibandronic acid on bone tissue Is based on its affinity for hydroxyapatite, which is part of the mineral matrix of bone.', 'Ibandronic acid inhibits osteoclast activity and reduces bone resorption and turnover.', 'In post menopausal women, it reduces the elevated rate of bone turnover, leading to, on average, a net gain in bone mass.']


In [12]:
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [13]:
print(clean_sentences[:5])

['for the use of a registered medical practitioner or a hospital or a laboratory only ', 'prescribing information inandronic acid tablets idrofos     dosage form  tablets composition  each film coated tablet contains  lbandronic acid monosodium monohydrate equivalentto ibandronic acid     mg excipients on clinical pharmacology  lbandronic acid is anitrogen containing bisphosphonate that inhibits osteoclast mediated bone resorption ', 'mechanism of action the action of ibandronic acid on bone tissue is based on its affinity for hydroxyapatite  which is part of the mineral matrix of bone ', 'ibandronic acid inhibits osteoclast activity and reduces bone resorption and turnover ', 'in post menopausal women  it reduces the elevated rate of bone turnover  leading to  on average  a net gain in bone mass ']


In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [16]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [17]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [18]:
print(clean_sentences[:5])

['use registered medical practitioner hospital laboratory', 'prescribing information inandronic acid tablets idrofos dosage form tablets composition film coated tablet contains lbandronic acid monosodium monohydrate equivalentto ibandronic acid mg excipients clinical pharmacology lbandronic acid anitrogen containing bisphosphonate inhibits osteoclast mediated bone resorption', 'mechanism action action ibandronic acid bone tissue based affinity hydroxyapatite part mineral matrix bone', 'ibandronic acid inhibits osteoclast activity reduces bone resorption turnover', 'post menopausal women reduces elevated rate bone turnover leading average net gain bone mass']


In [19]:
clean_doc = '. '.join(clean_sentences)
clean_doc += '.'

## Save Preprocessed File

In [130]:
with open("output.txt", "w") as f:
  f.write(clean_doc)

## Read Preprocessed File

In [ ]:
with open("output.txt", "r") as f:
  clean_doc = f.read()

In [20]:
doc = nlp(clean_doc)
for sents in doc.sents:
    print(sents.text)

use registered medical practitioner hospital laboratory.
prescribing information inandronic acid tablets idrofos dosage form tablets composition film coated tablet contains lbandronic acid monosodium monohydrate equivalentto ibandronic acid mg excipients clinical pharmacology lbandronic acid anitrogen containing bisphosphonate inhibits osteoclast mediated bone resorption.
mechanism action action ibandronic acid bone tissue based affinity hydroxyapatite part mineral matrix bone.
ibandronic acid inhibits osteoclast activity reduces bone resorption turnover.
post menopausal women reduces elevated rate bone turnover leading average net gain bone mass.
pharmacokinetics absorption oral ibandronic acid occurs upper gastrointestinal tract.
plasma concentrations increase ina dose linear manner mg oral intake increases nonlinearly dose.
following oral dosing time maximum observed plasma ibandronic acid concentrations ranged hours median hour fasted healthy postmenopausal women.
oral bioavailabil

In [78]:
#token.dep_, token.shape_, token.is_alpha, token.is_stop
from tabulate import tabulate
pos_res = []
text = []
tags = []
i = 0
for token in doc:
    #if token.text == '.':
      #i +=1
      #if i >10:
      #  break;
    if token.tag_ not in tags and token.is_punct == False:
      tags.append(token.tag_)
    if token.is_punct == False and token.text not in text:
      text.append(token.text)
    pos_res.append([token.text, token.tag_,token.is_sent_start,token.suffix_, token.is_punct])

In [104]:
###### test create ####
#######if you want to create for a specific number of sentences instead of fulldoc

from tabulate import tabulate
pos_res = []
text = []
tags = []
i = 0
for token in doc:
    if token.text == '.':
      i +=1
      if i >40:
        break;
    if token.tag_ not in tags and token.is_punct == False:
      tags.append(token.tag_)
    if token.is_punct == False and token.text not in text:
      text.append(token.text)
    pos_res.append([token.text, token.tag_,token.is_sent_start,token.suffix_, token.is_punct])

In [105]:

pos_res[0:5]    
print(tabulate([pos_res[i][:5] for i in range(20)], headers = ['TEXT', 'LEMMA', 'POS', 'TAG', 'DEP', 'SHAPE', 'ALPHA', 'STOP']))

TEXT          LEMMA    POS    TAG    DEP
------------  -------  -----  -----  -----
use           NN       True   use    False
registered    VBD             red    False
medical       NNP             cal    False
practitioner  NNP             ner    False
hospital      NNP             tal    False
laboratory    NNP             ory    False
.             .               .      True
prescribing   VBG      True   ing    False
information   NN              ion    False
inandronic    JJ              nic    False
acid          NN              cid    False
tablets       NNS             ets    False
idrofos       JJ              fos    False
dosage        NN              age    False
form          NN              orm    False
tablets       VBZ             ets    False
composition   NN              ion    False
film          NN              ilm    False
coated        VBD             ted    False
tablet        NN              let    False


# HMM

## Emmision and Probability Matrices

In [ ]:
print(text)

['use', 'registered', 'medical', 'practitioner', 'hospital', 'laboratory', 'prescribing', 'information', 'inandronic', 'acid', 'tablets', 'idrofos', 'dosage', 'form', 'composition', 'film', 'coated', 'tablet', 'contains', 'lbandronic', 'monosodium', 'monohydrate', 'equivalentto', 'ibandronic', 'mg', 'excipients', 'clinical', 'pharmacology', 'anitrogen', 'containing', 'bisphosphonate', 'inhibits', 'osteoclast', 'mediated', 'bone', 'resorption', 'mechanism', 'action', 'tissue', 'based', 'affinity', 'hydroxyapatite', 'part', 'mineral', 'matrix', 'activity', 'reduces', 'turnover', 'post', 'menopausal', 'women', 'elevated', 'rate', 'leading', 'average', 'net', 'gain', 'mass', 'pharmacokinetics', 'absorption', 'oral', 'occurs', 'upper', 'gastrointestinal', 'tract', 'plasma', 'concentrations', 'increase', 'ina', 'dose', 'linear', 'manner', 'intake', 'increases', 'nonlinearly', 'following', 'dosing', 'time', 'maximum', 'observed', 'ranged', 'hours', 'median', 'hour', 'fasted', 'healthy', 'post

In [106]:
#creating a dictionary for both tags and text of the form <data>:value. Not necessary but i found it convenient

text_dict = {j:i for i,j in enumerate(text)}
tag_dict = {j:i for i,j in enumerate(tags)}
#for i,j in enumerate(tags):
#  tag_dict[j] = i
#tag_dict = [OrderedDict(tags[i]:i for i in range(len(tags)))]
tag_dict

{'CC': 14,
 'CD': 16,
 'DT': 17,
 'FW': 15,
 'IN': 9,
 'JJ': 4,
 'JJR': 13,
 'JJS': 12,
 'LS': 18,
 'NN': 0,
 'NNP': 2,
 'NNS': 5,
 'RB': 10,
 'VB': 11,
 'VBD': 1,
 'VBG': 3,
 'VBN': 7,
 'VBP': 8,
 'VBZ': 6}

### Emission Matrix

In [107]:
def create_emission_matrix():
  emission_matrix = np.zeros((len(text),len(tags)))
  emission_matrix.fill(0)
  #key_list = list(my_dict.keys())
  #val_list = list(my_dict.values())

  '''
  basically we need to map the words to their tags.

  so we don't have to include the full stops so i've kept that check.

  so for each word in pos_res, we find the key it should be mapped to in the matrix and the tag that it should be mapped to
  We incriment the value in that cell

  same as doing text.index(i[0]) and tags.index(i[1])
  '''
  for i in pos_res:
    if i[0] != '.':
      try:
        emission_matrix[text_dict[i[0]]][tag_dict[i[1]]] += 1
      except:
        print("error ",i)
        break
  '''
  the following few lines basically deal with infinity and nan states that could occur due to division by zero
  '''
  with np.errstate(divide='ignore', invalid='ignore'):
    '''
    Now we need to average the columns to find the probability of each tag being a particular word.
    The below code does that:
    the code in the line below first does columnwise summation(axis=0) and averages
    '''
    emission_matrix =  emission_matrix/emission_matrix.sum(axis=0, keepdims=True)
    emission_matrix[emission_matrix == np.inf] = 0
    emission_matrix = np.nan_to_num(emission_matrix)
  return emission_matrix


In [108]:
emission_matrix = create_emission_matrix()
print(emission_matrix)
print(emission_matrix.shape)

[[0.00625    0.         0.         ... 0.         0.         0.        ]
 [0.         0.04761905 0.         ... 0.         0.         0.        ]
 [0.         0.         0.01219512 ... 0.         0.         0.        ]
 ...
 [0.         0.04761905 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00625    0.         0.         ... 0.         0.         0.        ]]
(280, 19)


### Transition Matrix

pos_res format text,tag,is_start,suffix.is_punct]

In [109]:
def create_transition_matrix():
  #we add +1 in both rows and columns to incorporate start and end state respectively
  transition_matrix = np.zeros(((len(tags)+1),(len(tags)+1)))
  transition_matrix.fill(0)
  #pointer points to those states
  pointer = len(tags)
  len_list = len(pos_res)


  for i in range(len(pos_res)):
    #since we cant find the i+1 of the last element, we keep this flag
    if i < len_list-1:
      
      #if it is the start of a sentence then increment the transition start_tag -> current_tag
      if pos_res[i][2] == True:
        transition_matrix[pointer][tag_dict[pos_res[i][1]]] +=1

      #if the next word is a '.' then that means that the current word is the end of the sentence. 
      #All special characters were removed during cleaning and only full stops were added while joining the sentences
      #sometimes due to the joining and formatting ofthe sentences, a case may come that there are two full stops consecutively.
      #to prevent that we make sure that both the current token and the next arent periods 
      elif pos_res[i+1][0] == '.':
        if pos_res[i][0] != '.':
            transition_matrix[tag_dict[pos_res[i][1]]][pointer] +=1

      #increment transition current tag -> next_tag
      else:
        if pos_res[i][0] != '.':
          transition_matrix[tag_dict[pos_res[i][1]]][tag_dict[pos_res[i+1][1]]] +=1

  #same thing as in emission
  with np.errstate(divide='ignore', invalid='ignore'):
    
    #again same concept as in what was done in emission but this time row wise (axis=1)
    transition_matrix =  transition_matrix/transition_matrix.sum(axis=1, keepdims=True)
    transition_matrix[transition_matrix == np.inf] = 0
    transition_matrix = np.nan_to_num(transition_matrix)
  return transition_matrix

In [110]:
transition_matrix = create_transition_matrix()
print(transition_matrix[0])
print(transition_matrix.shape)

[0.29605263 0.05921053 0.08552632 0.01315789 0.09868421 0.11842105
 0.04605263 0.07894737 0.01315789 0.03289474 0.03289474 0.
 0.00657895 0.         0.00657895 0.         0.         0.
 0.         0.11184211]
(20, 20)


In [111]:
for i in range(transition_matrix.shape[0]):
  print(transition_matrix[i])
  print(".......")

[0.29605263 0.05921053 0.08552632 0.01315789 0.09868421 0.11842105
 0.04605263 0.07894737 0.01315789 0.03289474 0.03289474 0.
 0.00657895 0.         0.00657895 0.         0.         0.
 0.         0.11184211]
.......
[0.22222222 0.         0.05555556 0.05555556 0.27777778 0.33333333
 0.         0.05555556 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
.......
[0.07246377 0.04347826 0.52173913 0.01449275 0.10144928 0.02898551
 0.01449275 0.01449275 0.         0.02898551 0.01449275 0.
 0.         0.01449275 0.         0.         0.01449275 0.
 0.         0.11594203]
.......
[0.2 0.  0.2 0.  0.6 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0. ]
.......
[0.62195122 0.         0.07317073 0.01219512 0.19512195 0.08536585
 0.         0.01219512 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
.......
[0.0625     0.08333333 0.14583333 0.0625  

In [ ]:
a= np.array([[0,0,0], [0,0,0], [0,0,0]])

x = np.array([1,2,3])
y = np.array([2,3,4])
z = np.multiply(x,y)*2
z

array([ 4, 12, 24])

In [ ]:
#x= np.array([[1,2,3], [4,5,6], [7,8,9], [5,3,2]])
#y = x[:-1,2]
x = np.array([1,2,3]).reshape((3,))
print(np.argmax(x))

2


In [ ]:
x = np.array([1,2,3])
y = np.array([2,3,4])
z = np.multiply(x,y)
z

array([ 2,  6, 12])

Reference Used (I did not have to be signed in to view these): 

Viterbi Algorithm: https://www.coursera.org/lecture/probabilistic-models-in-nlp/the-viterbi-algorithm-A5wSY

Initialization: https://www.coursera.org/lecture/probabilistic-models-in-nlp/viterbi-initialization-Q1Wuu

Forward Pass: https://www.coursera.org/lecture/probabilistic-models-in-nlp/viterbi-forward-pass-c6m8s

Backward Pass: https://www.coursera.org/lecture/probabilistic-models-in-nlp/viterbi-backward-pass-YqWsL

In [112]:
def viterbi(words):
  '''
  states = list to hold the most probable states during backward pass
  state_matrix = matrix to hold the maximum probability calculated from the previous state -> current state i.e. calculate (δ)
  tag_matrix = matrix that holds the index of the previous hidden state (or tag in the previous state) state that maximizes
               the probability of a word being a particular tag (or hidden state)
  '''
  states = []
  tag_matrix = np.zeros((len(tags),len(words)))
  state_matrix = np.zeros((len(tags),len(words)))


  ##Wherever you see ',:-1' it is to exclude the start or end the tags as they are not needed for the hidden states of each word 
  for key, word in enumerate(words):

    #if it is the first word i.e. start
    if key == 0:

      #there are no states before this we only calculate thetransition probability from start state to all the hidden states for the first word
      #elementwise multiplication between the transition matrix row of the start tag(transition probability from start to all other tags) and
      #the emission probability for the word wrt to all tags
      calc = np.multiply(transition_matrix[len(tags),:-1],  emission_matrix[text_dict[word],:]).reshape((len(tags),1))
      state_matrix[:,0] = calc.ravel()
      #print("...................")
      #print(calc)

    else:

      #if its not start then we need to calculate the max probability from all possible hidden states for previous word
      #with the current hidden state
      for tag in tags:
        tag_key = tag_dict[tag]

        #basically we do elementwise multiplication
        #for all hidden states of previous word we multiply
        # transition probability of previous state to current, the maximum probability of the previous hidden state under consideration(δ) and
        #emission probaility of a particular word from the hidden state
        #refer to the forward pass video from above for the exact formula

        calc = np.multiply(state_matrix[:, key-1],transition_matrix[:-1,tag_key]) * emission_matrix[text_dict[word]][tag_key]
        
        state_matrix[tag_key][key] = np.amax(calc)   #find the maximum probability
        tag_matrix[tag_key][key] = np.argmax(calc)   #find the previous state that maximizes this one

  
  #finding the probability that a state would be an end state. We take the argmax for this to
  #find the maximum likelihood state for the last word
  max_state = np.argmax(np.multiply(transition_matrix[:-1, len(tags)], state_matrix[:,-1])) 
  
  states.append(tags[int(max_state)])

  #refer backward state video
  #so now we do the backward pass where we iterate through tag_matrix backward.
  #We find the max state for a particular word. That is the previous hidden state that maximizes the current.
  #This way we move backward and keep storing the maximum likelihood tag in states
  for i in reversed(range(1,len(words))):
    max_state = int(tag_matrix[max_state][i])
    states.append(tags[max_state])

  #print(state_matrix)
  #print(tag_matrix)
  #print(states)

  #reverse states because we've appended it from last to first
  return states[::-1]
  
  


In [123]:
###Choose sentence to run it on

test_doc = nlp(clean_sentences[1])
test_words = []
test_tags = []
for token in test_doc:
  test_words.append(token.text)
  test_tags.append(token.tag_)
print(test_words)
print(test_tags)
ans = viterbi(test_words)
print(ans)

['prescribing', 'information', 'inandronic', 'acid', 'tablets', 'idrofos', 'dosage', 'form', 'tablets', 'composition', 'film', 'coated', 'tablet', 'contains', 'lbandronic', 'acid', 'monosodium', 'monohydrate', 'equivalentto', 'ibandronic', 'acid', 'mg', 'excipients', 'clinical', 'pharmacology', 'lbandronic', 'acid', 'anitrogen', 'containing', 'bisphosphonate', 'inhibits', 'osteoclast', 'mediated', 'bone', 'resorption']
['VBG', 'NN', 'JJ', 'NN', 'NNS', 'JJ', 'NN', 'NN', 'VBZ', 'NN', 'NN', 'VBD', 'NN', 'VBZ', 'JJ', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'VBZ', 'JJ', 'NN', 'NNP', 'NNP', 'NNP', 'VBG', 'NNP', 'NNS', 'NNP', 'VBN', 'NN', 'NN']
['VBG', 'NN', 'JJ', 'NN', 'VBZ', 'JJ', 'NN', 'NN', 'VBZ', 'NN', 'NN', 'VBD', 'NN', 'VBZ', 'JJ', 'NN', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'VBZ', 'JJ', 'NN', 'NNP', 'NNP', 'NNP', 'VBG', 'NNP', 'NNS', 'NNP', 'VBN', 'NN', 'NN']


In [124]:
#tabulating for comparison

from tabulate import tabulate

formatted_ans = []
for i in range(len(test_words)):
  formatted_ans.append([test_words[i], ans[i], test_tags[i]])

print(tabulate(formatted_ans, headers= ["Word", "Predicted Tag", "Actual Tag"]))

Word            Predicted Tag    Actual Tag
--------------  ---------------  ------------
prescribing     VBG              VBG
information     NN               NN
inandronic      JJ               JJ
acid            NN               NN
tablets         VBZ              NNS
idrofos         JJ               JJ
dosage          NN               NN
form            NN               NN
tablets         VBZ              VBZ
composition     NN               NN
film            NN               NN
coated          VBD              VBD
tablet          NN               NN
contains        VBZ              VBZ
lbandronic      JJ               JJ
acid            NN               NNP
monosodium      NNP              NNP
monohydrate     NNP              NNP
equivalentto    NNP              NNP
ibandronic      NNP              NNP
acid            NNP              NNP
mg              NNP              NNP
excipients      VBZ              VBZ
clinical        JJ               JJ
pharmacology    NN               